In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool

In [2]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""
    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [33]:
class GCNByHand(nn.Module):
    def __init__(self, N, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.out = torch.nn.Linear(N, out_channels, bias=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        # print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        # print("x after second layer\n", x)
        x = torch.relu(x)
        # x = torch.transpose(x, 0, 1)
        print(x)
        print(self.out.bias, self.out.weight, self.out.weight.shape)
        x = self.out(x)
        print(x)
        x = torch.sigmoid(x)

        # print("x after output layer\n", x)
        # x = global_mean_pool(x, torch.zeros(x.size(1)).long())  # all are from graph 0, single graph
        return x

In [34]:
num_nodes = 3       # N
num_features = 2    # D
num_labels = 1      # O

# the following needs to be replaced by some node embeddings
x = torch.randn((num_features, num_nodes)) # D x N

# print("x", x, x.shape)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(N=num_nodes, in_channels=num_features, out_channels=num_labels)

out = model(x, A)   # D x O

print("output")
print(out)

tensor([[0.4092, 0.2457, 0.4092],
        [0.8500, 0.5871, 0.8500]], grad_fn=<ReluBackward0>)
Parameter containing:
tensor([-0.0747], requires_grad=True) Parameter containing:
tensor([[-0.2958,  0.2407, -0.3674]], requires_grad=True) torch.Size([1, 3])
tensor([[-0.2870],
        [-0.4972]], grad_fn=<AddmmBackward0>)
output
tensor([[0.4287],
        [0.3782]], grad_fn=<SigmoidBackward0>)


In [39]:
x = np.array([[0.4092, 0.2457, 0.4092],
        [0.8500, 0.5871, 0.8500]])

w = np.array([[-0.2958,  0.2407, -0.3674]])

b = np.array([-0.0747])

In [40]:
b + x @ w.transpose()

array([[-0.28694145],
       [-0.49710503]])

In [64]:
X = np.array([
    [
        [1, 1]
    ],
    [
        [1, 1]
    ],
    [
        [1, 1]
    ]
])

X, X.shape

(array([[[1, 1]],
 
        [[1, 1]],
 
        [[1, 1]]]),
 (3, 1, 2))

In [65]:
Y = np.array([
    [
        [1], 
        [1], 
    ],
    [
        [1], 
        [2], 
    ]
    ,
    [
        [1], 
        [2], 
    ]
])
Y, Y.shape

(array([[[1],
         [1]],
 
        [[1],
         [2]],
 
        [[1],
         [2]]]),
 (3, 2, 1))

In [66]:
Z = X @ Y
Z, Z.shape

(array([[[2]],
 
        [[3]],
 
        [[3]]]),
 (3, 1, 1))